In [1]:
import os

In [4]:
%pwd

'e:\\project\\dsproject'

In [3]:
os.chdir("../")
%pwd

'e:\\project\\dsproject'

We need to update.config

``` python

artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

--Now you can see this variable are metioned in yaml file above.
--other point to mention is here we are using dataclass, this is diff from normal class method you dont need to give self parameter over here

Next step we are going to updata config manager.
but where you have define this CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH that seeing below code ?
src>constant>--init--.py

```python
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")


In [6]:
from src.dsproject.constants import *
from src.dsproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

Now second function is data ingetion

In [8]:
import os
import urllib.request as request
from src.dsproject import logger
import zipfile

In [9]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-24 01:49:59,407: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-24 01:49:59,418: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-24 01:49:59,421: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-24 01:49:59,423: INFO: common: created directory at: artifacts]
[2025-02-24 01:49:59,426: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-24 01:50:01,221: INFO: 251335794: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7972:2A8F9A:1EE1FC:5F221B:67BB82EF
Accept-Ranges: bytes
Date: Sun, 23 